# Практическая работа №12

## Проверка GPU

In [1]:
!nvidia-smi

Sun Dec 21 19:14:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Настройка окружения

In [2]:
!apt-get -qq update
!apt-get -qq install -y build-essential python3-dev
!pip -q install pycuda

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package javascript-common.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../javascript-common_11+nmu1_all.deb ...
Unpacking javascript-common (11+nmu1) ...
Selecting previously unselected package libjs-underscore.
Preparing to unpack .../libjs-underscore_1.13.2~dfsg-2_all.deb ...
Unpacking libjs-underscore (1.13.2~dfsg-2) ...
Selecting previously unselected package libjs-sphinxdoc.
Preparing to unpack .../libjs-sphinxdoc_4.3.2-1_all.deb ...
Unpacking libjs-sphinxdoc (4.3.2-1) ...
Selecting previously unselected package python3.10-dev.
Preparing to unpack .../python3.10-dev_3.10.12-1~22.04.12_amd64.deb ...
Unpacking python3.10-dev (3.10.12-1~22.04.12) ...
Selecting previously unselected package python3-dev.
Preparing to un

In [3]:
import numpy as np
import pycuda.autoinit  # автоматически создаёт CUDA context
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import math
import time

## Создание ядра

In [4]:
kernel_code = r"""
#include <math.h>

extern "C" __global__ void fill_array(float *out, int n) {
    int i = (int)(blockIdx.x * blockDim.x + threadIdx.x);
    if (i < n) {
        float sign = (i & 1) ? -1.0f : 1.0f;  // (-1)^i
        float fi = (float)i;
        out[i] = sign * fi * sinf(fi) * cosf(fi);
    }
}
"""

mod = SourceModule(kernel_code)
fill_array = mod.get_function("fill_array")

## Запуск задачи

In [23]:
N = 10_000_000

out = np.empty(N, dtype=np.float32)

threads = 256
blocks = (N + threads - 1) // threads

# запуск kernel
fill_array(
    cuda.Out(out),
    np.int32(N),
    block=(threads, 1, 1),
    grid=(blocks, 1),
)

print("Готово. Первые 10 элементов:", out[:10])

Готово. Первые 10 элементов: [ 0.        -0.4546487 -0.7568025  0.4191232  1.9787166  1.3600527
 -1.6097187 -3.4671257 -1.1516132  3.3794427]


## Замер времени GPU и CPU

### GPU

In [24]:
cuda.Context.synchronize()
t0 = time.time()

fill_array(
    cuda.Out(out),
    np.int32(N),
    block=(threads, 1, 1),
    grid=(blocks, 1),
)

cuda.Context.synchronize()
gpu_time = time.time() - t0
print("GPU time (s):", gpu_time)

GPU time (s): 0.012094736099243164


### CPU

In [25]:
t0 = time.time()

cpu_out = np.empty(N, dtype=np.float32)
for i in range(N):
    cpu_out[i] = ((-1)**i) * i * math.sin(i) * math.cos(i)

cpu_time = time.time() - t0
print("CPU time (s):", cpu_time)

CPU time (s): 6.664976358413696
